# Análise Exploratória de Dados – Job Postings

Este notebook apresenta uma análise exploratória do dataset **fake_real_job_postings**, incluindo:

- Estatísticas descritivas
- Detecção de outliers
- Boxplots
- Distribuições e regressões
- Matriz de correlação

O objetivo é identificar padrões, anomalias e relações relevantes entre as variáveis,
servindo como base para análises estatísticas e modelos preditivos.


In [14]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, length, to_date, countDistinct
import plotly.express as px
import pandas as pd


In [4]:
spark = SparkSession.builder \
    .appName("Analise_Raw_Layer") \
    .master("local[*]") \
    .getOrCreate()


In [5]:
df_raw = spark.read.csv("data_raw.csv", header=True, inferSchema=True)
print("Dados carregados. Total de linhas:", df_raw.count())

Dados carregados. Total de linhas: 3000


## Gráfico 1: Completude dos Dados
Apresentando o percentual de registros preenchidos (não nulos) em relação ao total da base. As colunas estão ordenadas do menor para o maior nível de preenchimento, facilitando a identificação de campos com maior quantidade de dados ausentes.

In [2]:
total_rows = df_raw.count()

completeness_exprs = []

for c in df_raw.columns:

    expressao = count(when(col(c).isNotNull(), c))

    percentual = (expressao / total_rows * 100).alias(c)
    completeness_exprs.append(percentual)

quality_df = df_raw.select(*completeness_exprs).toPandas().transpose().reset_index()
quality_df.columns = ['Coluna', 'Preenchimento']
quality_df = quality_df.sort_values('Preenchimento')

fig1 = px.bar(quality_df, x='Preenchimento', y='Coluna', orientation='h',
              title='1. Completude: Percentual de Dados Presentes',
              color='Preenchimento', color_continuous_scale='RdYlGn', text_auto='.1f')
fig1.update_layout(height=600)
fig1.show()

## Gráfico 2: Cardinalidade
Apresentando as 15 colunas com maior quantidade de valores únicos. A escala logarítmica no eixo X é utilizada para facilitar a visualização de diferenças de ordem de grandeza entre as colunas, auxiliando na identificação de campos com alta variabilidade ou possíveis identificadores únicos.

In [15]:
card_df = df_raw.select([countDistinct(c).alias(c) for c in df_raw.columns]).toPandas().transpose()
card_df.columns = ['Valores Unicos']
card_df = card_df.sort_values('Valores Unicos', ascending=False).head(15)

fig2 = px.bar(card_df, x='Valores Unicos', y=card_df.index, orientation='h',
              title='2. Cardinalidade: Top 15 Colunas com Maior Variedade',
              text_auto=True)
fig2.update_xaxes(type="log", title="Qtd. Valores Únicos (Escala Log)")
fig2.show()

 ## Gráfico 3: Distribuição de Tipos de Dados
apresentando o perfil técnico do dataset e auxiliando na identificação de possíveis problemas de tipagem e na preparação de transformações para análises posteriores.

In [26]:
dtypes_df = pd.DataFrame(df_raw.dtypes, columns=['Coluna', 'Tipo'])
type_counts = dtypes_df['Tipo'].value_counts().reset_index()
type_counts.columns = ['Tipo', 'Contagem']

fig3 = px.pie(type_counts, values='Contagem', names='Tipo',
              title='3. Perfil Técnico: Tipos de Dados Ingeridos')
fig3.show()

## Gráfico 4: Matriz de Correlação de Nulos
utilizada para identificar padrões de ausência de dados e relações entre colunas que apresentam falhas de preenchimento de forma conjunta ou dependente.

In [25]:
total_rows = df_raw.count()

null_exprs = [count(when(col(c).isNull(), c)).alias(c) for c in df_raw.columns]
null_counts = df_raw.select(null_exprs).collect()[0].asDict()

data_list = []
for coluna, nulos in null_counts.items():
    if nulos > 0:
        percentual = (nulos / total_rows) * 100
        label_texto = f"{nulos:,} ({percentual:.1f}%)"

        data_list.append({
            "Coluna": coluna,
            "Nulos": nulos,
            "Total": total_rows,
            "%": percentual,
            "Label": label_texto
        })

df_missing = pd.DataFrame(data_list).sort_values(by="Nulos", ascending=True)

display(df_missing.sort_values(by="Nulos", ascending=False))

,Coluna,Nulos,Total,%,Label
2,fraud_reason,1528,3000,50.933333,"1,528 (50.9%)"
0,company_name,1472,3000,49.066667,"1,472 (49.1%)"
1,company_website,1472,3000,49.066667,"1,472 (49.1%)"


In [23]:
df_plot = df_missing.tail(15)

fig = px.bar(
    df_plot,
    x="Nulos",
    y="Coluna",
    orientation='h',
    text="Label",
    title="Top Colunas com Dados Faltantes",
    color="Nulos",
    color_continuous_scale="RdBu_r"
)

fig.update_traces(
    textposition='outside',
    cliponaxis=False
)

fig.update_layout(
    xaxis_title="Quantidade de Valores Nulos",
    yaxis_title=None,
    coloraxis_showscale=False,
    height=500,
    margin=dict(r=100)
)

fig.show()

## Gráfico 5: Volume de Texto (Caracteres)
medida pelo número de caracteres, utilizada para avaliar a consistência, integridade e presença de outliers em campos descritivos do dataset.

In [21]:
text_metrics = df_raw.select(
    length(col("job_description")).alias("Descricao"),
    length(col("requirements")).alias("Requisitos")
).sample(0.1, seed=42).toPandas()

fig5 = px.box(pd.melt(text_metrics), x="variable", y="value",
              title='5. Integridade do Texto: Quantidade de Caracteres',
              labels={'value':'Tamanho', 'variable':'Campo'})
fig5.show()

## Gráfico 6: Consistência: Top 10 Indústrias
utilizada para avaliar a consistência e concentração do campo categórico industry, identificando possíveis vieses e necessidades de padronização.

In [22]:
top_ind = df_raw.groupBy("industry").count().orderBy(col("count").desc()).limit(10).toPandas()

fig6 = px.bar(top_ind, x='count', y='industry', orientation='h',
              title='6. Consistência de Domínio: Top Indústrias', text_auto=True)
fig6.update_layout(yaxis={'categoryorder':'total ascending'})
fig6.show()

## Gráfico 7: Consistência: Top 10 Localizações
utilizada para avaliar a consistência do campo geográfico location e identificar possíveis concentrações, vieses ou problemas de padronização.

In [23]:
top_loc = df_raw.groupBy("location").count().orderBy(col("count").desc()).limit(10).toPandas()

fig7 = px.bar(top_loc, x='count', y='location', orientation='h',
              title='7. Consistência Geográfica: Top Localizações',
              color_discrete_sequence=['teal'], text_auto=True)
fig7.update_layout(yaxis={'categoryorder':'total ascending'})
fig7.show()

# Gráfico 8: Distribuição Numérica (Experiência)
utilizada para validar a consistência dos valores numéricos, identificar outliers e compreender o perfil de senioridade predominante no dataset.

In [24]:
exp_dist = df_raw.select("required_experience_years").toPandas()

fig8 = px.histogram(exp_dist, x="required_experience_years", nbins=15,
                    title='8. Validação Numérica: Anos de Experiência',
                    labels={'required_experience_years':'Anos'})
fig8.show()

## Gráfico 9: Timeline da Ingestão
apresentando o volume diário de registros ao longo do tempo e auxiliando na identificação de padrões, picos e possíveis falhas no processo de ingestão dos dados.

In [16]:

date_dist = df_raw.select(to_date(col("posting_date")).alias("data")) \
                  .groupBy("data").count().orderBy("data").toPandas()

fig9 = px.line(date_dist, x='data', y='count', markers=True,
               title='9. Distribuição Temporal das Vagas')
fig9.show()

## Gráfico 10: Validação dos campos Binários
Distribuição dos valores binários (0 e 1) nos campos booleanos do dataset, utilizada para avaliar o balanceamento das flags e identificar possíveis vieses ou problemas de preenchimento.
Nos campos binários, o valor 0 representa a ausência da característica (falso), enquanto o valor 1 indica a presença (verdadeiro).

In [9]:
flags_df = df_raw.select("telecommuting", "has_logo", "is_fake").toPandas()
flags_melt = pd.melt(flags_df)

fig10 = px.histogram(flags_melt, x="variable", color="value", barmode="group",
                     title='10. Balanceamento de Flags (0 vs 1)',
                     labels={'variable':'Campo', 'value':'Valor (0/1)'})
fig10.show()